In [17]:
import numpy as np

In [18]:
def REF(M, verbose=False):
    def sortTransform(M, verbose = verbose):

        newM = M.copy()
        
        for k in reversed(range(newM.shape[1])):
            # Create a boolean matrix, where 1 is values above 0.
            boolM = np.where(np.isclose(newM, 0), 0 ,1)
            # Sort the matrix by the boolean matrix, this preserves the order as best as possible, only switching the rows when there is a non-zero value below a zero.
            newIndeces = np.argsort(-boolM[:,k])
            # Sort the matrix
            newM = newM[newIndeces]
            
        return newM

    def refTransform(M, verbose = verbose):

        if M.shape[0] == 1 and M.shape[1] == 1:
            print(f"Matrix is {M.shape[0]}x{M.shape[1]}") if verbose == True else None
            M = M / M if M > 1 else M
            return M

        elif M.shape[0] == 0 or M.shape[1] == 0:
            print(f"Matrix is {M.shape[0]}x{M.shape[1]}") if verbose == True else None
            return M
        
        elif np.all(M==0):
            print(f"Matrix is is all zeros") if verbose == True else None
            return M

        else:
            
            #Sort Matrix
            A = sortTransform(M) if np.nonzero(M[:,0])[0].size < len(M[:,0]) else M.copy()
            print(f"Sorting Matrix \n {str(A)}") if verbose == True else None
            # Divide row by its first non-zero element
            for row in range(M.shape[0]):
                A[row] = A[row] / A[row,0] if A[row,0] else A[row]
                print(f"Dividing row {row} by {A[row,0]}") if verbose == True else None
                print(f"The result is {str(A)}") if verbose == True else None
            # Subtract the row from all other rows
            A[1:] -= A[0] * A[1:,0:1]
            print(f"Subtracting row {0} from all other rows.\n tbe result is\n {str(A)}") if verbose == True else None
            # Check if there are any other non-zero elements below the first row  
            k = 1 if np.nonzero(A[:,0])[0].size > 0 else 0 
            # If there are, repeat the process by taking a subset of the matrix, k rows below the first (determined by whether there are non-zero elements below the first row)
            B = A[k:,1:]
            print(f"Sending the subset of the matrix\n {str(B)}") if verbose == True else None
            # Recursively call the function
            B = refTransform(B)
            # Update the matrix
            A[k:,1:] = B
            print(f"The resulting matrix is\n {str(A)}") if verbose == True else None
            
            return A
        
    newM = refTransform(M)
    
    return newM

In [19]:
M =np.array([[1., 2., 3.,4],
          [5., 0., 2.,3],
          [1., 4., 5.,6]])
REF(M,verbose = True)

Sorting Matrix 
 [[1. 2. 3. 4.]
 [5. 0. 2. 3.]
 [1. 4. 5. 6.]]
Dividing row 0 by 1.0
The result is [[1. 2. 3. 4.]
 [5. 0. 2. 3.]
 [1. 4. 5. 6.]]
Dividing row 1 by 1.0
The result is [[1.  2.  3.  4. ]
 [1.  0.  0.4 0.6]
 [1.  4.  5.  6. ]]
Dividing row 2 by 1.0
The result is [[1.  2.  3.  4. ]
 [1.  0.  0.4 0.6]
 [1.  4.  5.  6. ]]
Subtracting row 0 from all other rows.
 tbe result is
 [[ 1.   2.   3.   4. ]
 [ 0.  -2.  -2.6 -3.4]
 [ 0.   2.   2.   2. ]]
Sending the subset of the matrix
 [[-2.  -2.6 -3.4]
 [ 2.   2.   2. ]]
Sorting Matrix 
 [[-2.  -2.6 -3.4]
 [ 2.   2.   2. ]]
Dividing row 0 by 1.0
The result is [[1.  1.3 1.7]
 [2.  2.  2. ]]
Dividing row 1 by 1.0
The result is [[1.  1.3 1.7]
 [1.  1.  1. ]]
Subtracting row 0 from all other rows.
 tbe result is
 [[ 1.   1.3  1.7]
 [ 0.  -0.3 -0.7]]
Sending the subset of the matrix
 [[-0.3 -0.7]]
Sorting Matrix 
 [[-0.3 -0.7]]
Dividing row 0 by 1.0
The result is [[1.         2.33333333]]
Subtracting row 0 from all other rows.
 tbe result

array([[1.        , 2.        , 3.        , 4.        ],
       [0.        , 1.        , 1.3       , 1.7       ],
       [0.        , 0.        , 1.        , 2.33333333]])

In [20]:
test_cases = [{'name': 'check_null_matrix',
  'input': {'A': np.array([[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]),
   'B': np.array([[0],
          [0],
          [0]])},
  'expected': np.array([[0, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]])},
 {'name': 'check_identity_matrix',
  'input': {'A': np.array([[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.],
          [0., 0., 0.]]),
   'B': np.array([[0.],
          [0.],
          [0.],
          [1.]])},
  'expected': np.array([[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]])},
 {'name': 'check_matrix_1',
  'input': {'A': np.array([[1., 2., 3.],
          [5., 0., 2.],
          [1., 4., 5.]]),
   'B': np.array([[4.],
          [3.],
          [6.]])},
  'expected': np.array([[ 1.        ,  2.        ,  3.        ,  4.        ],
         [-0.        ,  1.        ,  1.3       ,  1.7       ],
         [-0.        , -0.        ,  1.        ,  2.33333333]])},
 {'name': 'check_matrix_2',
  'input': {'A': np.array([[ 1.,  5.,  6.],
          [ 3.,  1.,  4.],
          [ 2., -4., -2.]]),
   'B': np.array([[ 9.],
          [ 4.],
          [-5.]])},
  'expected': np.array([[ 1.        ,  5.        ,  6.        ,  9.        ],
         [-0.        ,  1.        ,  1.        ,  1.64285714],
         [ 0.        ,  0.        ,  0.        ,  0.        ]])}]

In [21]:
for test_case in test_cases:
    print(f"Running test case {test_case['name']}...")
    M = np.hstack((test_case['input']['A'], test_case['input']['B']))
    newM = REF(M)
    #newM=row_echelon(M)
    if np.allclose(newM, test_case['expected']):
        print("Test passed!")
    else:
        print("Test failed!")
        print(f"Expected: {str(test_case['expected'])}\nOutput: {str(newM)}")

Running test case check_null_matrix...
Test passed!
Running test case check_identity_matrix...
Test passed!
Running test case check_matrix_1...
Test passed!
Running test case check_matrix_2...
Test passed!
